In [1]:
!pip install transformers

In [2]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Sentiment analysis/project 2024/Dataset-20231210T071705Z-001/Dataset/train.csv")
df = df.drop('Unnamed: 0', axis=1)
print(df.head())

                                              review sentiment
0  One reviewer mentioned after watching Oz episo...  positive
1  A wonderful little production The filming tech...  positive
2  I thought wonderful way spend time hot summer ...  positive
3  Basically s family little boy Jake think s zom...  negative
4  Petter Mattei Love Time Money visually stunnin...  positive


In [3]:
def sentiment(text):
  if text=='positive':
    return 1
  else:
    return 0
df['sentiment']=df['sentiment'].apply(sentiment)
df.head()

,review,sentiment
0,One reviewer mentioned after watching Oz episo...,1
1,A wonderful little production The filming tech...,1
2,I thought wonderful way spend time hot summer ...,1
3,Basically s family little boy Jake think s zom...,0
4,Petter Mattei Love Time Money visually stunnin...,1


In [4]:
!pip install sentence-transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.7 MB/s eta 0:00:00


In [5]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
import torch

In [6]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
sentence_embeddings = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
from sentimentBERT import *
from model_lr import *

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [14]:
dataset = SentimentDataset(df, tokenizer)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
# Define the input and output dimensions
input_dim = 512 * 768 # product of the remaining dimensions
output_dim = 1 # binary classification

# Create an instance of the model
model = LogisticRegression(input_dim, output_dim)
model=model.to(device)

# Define the loss function
criterion = nn.BCEWithLogitsLoss()

# Define the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [17]:
from transformers import AutoTokenizer, AutoModel

In [18]:
#Load the SentimentBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
# Load the SentimentBERT model
bert_model = AutoModel.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


In [ ]:
# Load the data
data = pd.read_csv("/content/drive/MyDrive/Sentiment analysis/project 2024/Dataset-20231210T071705Z-001/Dataset/train.csv")

# Create a dataloader
dataloader = DataLoader(data, batch_size=128, shuffle=True)

In [20]:
# Train the model
num_epochs = 2

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}")

    for inputs, labels in dataloader:
        optimizer.zero_grad()
        labels = labels.unsqueeze(1).float()
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        print(f"Batch Loss: {loss.item():.4f}")

Epoch 1


KeyboardInterrupt: 

In [ ]:
# save the trained model
torch.save(model.state_dict(), 'model_bert_large.pth')

In [ ]:
model.eval()


In [ ]:
#Reading test file
import pandas as pd
df1 = pd.read_csv("/content/drive/MyDrive/0000/val.csv")
del df1['Unnamed: 0']
print(df1.head())

In [ ]:
def sentiment(text):
  if text=='positive':
    return 1
  else:
    return 0
df1['sentiment']=df1['sentiment'].apply(sentiment)
df1.head()

In [ ]:
dataset1 = SentimentDataset(df1, tokenizer)
dataloader1 = DataLoader(dataset1, batch_size=128, shuffle=True)

In [ ]:
!pip install torchvision


In [ ]:
!pip install -U pillow


In [ ]:
pip install -U torchvision


In [ ]:
import torch
# Define the path to your saved model file
model_path = '/model_bert_large.pth'

In [ ]:
# Load the saved model state dictionary from the file onto the GPU
state_dict = torch.load(model_path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
state_dict = {k: v.to(device) for k, v in state_dict.items()}

In [ ]:
# Load the saved model state dictionary into the model
model.load_state_dict(state_dict)

In [ ]:
model = model.to(device)


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt

model.eval()



In [ ]:
# Initialize variables to store the true labels and predicted probabilities
true_labels = []
pred_probs = []

# Iterate over the test dataset and collect the true labels and predicted probabilities
with torch.no_grad():
    for i, batch in enumerate(dataloader1, start=1):
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        print("Batch",i,"started")
        outputs = model(inputs)
        probs = torch.sigmoid(outputs)
        preds = torch.round(probs)

        true_labels.extend(labels.cpu().numpy())
        pred_probs.extend(probs.cpu().numpy())
        print("Batch",i,"completed")
# Compute various evaluation metrics based on the true labels and predicted probabilities
accuracy = accuracy_score(true_labels, np.round(pred_probs))
precision = precision_score(true_labels, np.round(pred_probs))
recall = recall_score(true_labels, np.round(pred_probs))
f1 = f1_score(true_labels, np.round(pred_probs))
auc_roc = roc_auc_score(true_labels, pred_probs)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC ROC: {auc_roc:.4f}")

In [ ]:
# Compute the ROC curve and the AUC
fpr, tpr, thresholds = roc_curve(true_labels, pred_probs)

In [ ]:
# Plot the ROC curve
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

In [ ]:
# Compute the confusion matrix
tn, fp, fn, tp = confusion_matrix(true_labels, np.round(pred_probs)).ravel()